In [8]:
import numpy as np
import pandas as pd
import xarray as xr
import random
import time
import os
from pathlib import Path

In [9]:
def fetch_data():
    '''
    reads the raw data
    '''
    dir_path = Path().resolve().as_posix()
    # print(dir_path)
    df = pd.read_json("https://raw.githubusercontent.com/acycliq/issplus/master/dashboard/data/img/default/json/iss.json")
    GeneExp = np.load(dir_path + '/data_preprocessed/GeneExp.npy')
    genes = [line.rstrip("\n''") for line in open(dir_path + '/data_preprocessed/genes.csv')]
    ctc = [line.rstrip("\n''") for line in open(dir_path + '/data_preprocessed/cell_to_class_map.csv')]

    # Rename PC.CA2 to PC.Other1 and PC.CA3 to PC.Other2
    ctc = ['PC.Other1' if x == 'PC.CA2' else x for x in ctc]
    ctc = ['PC.Other2' if x == 'PC.CA3' else x for x in ctc]

    ge = xr.DataArray(GeneExp, coords=[genes, ctc], dims=['Genes', 'Class'])
    return df, ge

In [10]:
def best_class(df):
    '''
    Returns a list with the names of all the optimal/best classes
    '''
    class_name = df['ClassName']
    prob = df['Prob']
    out = [class_name[n][np.argmax(prob[n])] for n in range(class_name.shape[0])]
    return out

In [11]:
def draw_gene_expression(df, ge):
    '''
    Main purpose is to make gene expression 2d arrays of dimension
    num_of_genes by num_of_classes by randomly sampling from the main
    (full) expression matrix (which is of size num_of_genes by num_of_cells)
    '''
    # cell class (unique and ranked alphabetically)
    best_classes = sorted(set(df['best_class']))
    class_list = ge.Class.values.tolist()
    N = len(best_classes)
    M = ge.shape[0]
    out = {'cid': [],
           'Cell_Num': [],
           'X': [],
           'Y': [],
           'class_name': [],
           'gene_name': ge.Genes.values.tolist(),
           'col': [],
           'GeneExp': np.empty([M, N], dtype=np.int32)
           }
    for i in range(N):
        # select a class
        bc = best_classes[i]
#         print(bc)

        # carve out data only relevant to the selected class
        class_df = df[df['best_class'] == bc]

        # randomly select a cell of that specific class
        cid = random.choice(class_df.index)
        temp = class_df.loc[cid]

        # keep the data for that particular cell to a dictionary
        out['cid'].append(cid)
        out['Cell_Num'].append(temp['Cell_Num'])  # This is 1-based, not 0-based
        out['X'].append(temp['X'])
        out['Y'].append(temp['Y'])
        out['class_name'].append(temp['best_class'])
#         start = time.time()
        mask = [i for i in range(len(class_list)) if class_list[i] == bc]
#         print(time.time() - start)
        col = random.choice(mask)
        out['col'].append(col)
        out['GeneExp'][:, i] = ge[:, col]

    return out

Sort them:

In [12]:
def thinner(data):
    '''
    shrink the gene expressions towards zero
    '''
    p = 0.1
    mat = data['GeneExp']
    rnd = np.random.binomial(mat, p)
    data['GeneExp'] = rnd
    return data

In [13]:
def position_genes(data):
    '''
    place the genes on the plane, centered around the corresponding cell center
    where the locations are distributed randomly (and independedly) with st dev r
    '''
    r = 8.8673
    u = np.random.normal(0, r, data['GeneExp'].shape)
    v = np.random.normal(0, r, data['GeneExp'].shape)
    _xCoord = (data["X"]+u)*(data['GeneExp'] > 0)
    _yCoord = (data["Y"]+v)*(data['GeneExp'] > 0)

    xCoord = pd.DataFrame(_xCoord,
                          columns=data['class_name'],
                          index=data['gene_name']
                          )
    yCoord = pd.DataFrame(_yCoord,
                          columns=data['class_name'],
                          index=data['gene_name']
                          )

    # print('in position')
    return xCoord, yCoord

In [14]:
maxIter = 10 #sample size
xCoord = []
yCoord = []

# Fetch the data
raw_data, gene_expression = fetch_data()

# for each cell find its most likely cell class
bc = best_class(raw_data)

# stick it at the end of the dataframe
raw_data['best_class'] = bc

# remove cells belonging to the Zero class
nonZero = raw_data['best_class'] != 'Zero'
raw_data = raw_data[nonZero]

start = time.time()
for i in range(maxIter):
    sample = draw_gene_expression(raw_data, gene_expression)

    sample = thinner(sample)

    _xCoord, _yCoord = position_genes(sample)
    xCoord.append(_xCoord)
    yCoord.append(_yCoord)
    
print('Done!')
print('finished %d iterations in %4.2f secs' % (maxIter, time.time()-start))

Done!
finished 10 iterations in 0.88 secs


In [15]:
xCoord[0].shape

(92, 59)

In [16]:
xCoord[0].head()

,Astro.1,Astro.2,Astro.3,Astro.4,Astro.5,Cacna2d1.Lhx6.Reln,Cacna2d1.Lhx6.Vwa5a,Cacna2d1.Ndnf.Cxcl14,Cacna2d1.Ndnf.Npy,Cacna2d1.Ndnf.Rgs10,...,Sst.Erbb4.Crh,Sst.Erbb4.Rgs10,Sst.Npy.Cort,Sst.Npy.Mgat4c,Sst.Npy.Serpine2,Sst.Npy.Zbtb20,Sst.Pnoc.Calb1.Igfbp5,Sst.Pnoc.Calb1.Pvalb,Sst.Pnoc.Pvalb,Vip.Crh.C1ql1
3110035E14Rik,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
6330403K07Rik,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,8200.357407,0.0,0.0,...,11771.037811,0.0,0.000000,13331.067375,7181.596109,0.0,12517.394178,0.0,0.0,0.0
Adgrl2,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,8750.449889,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
Aldoc,6847.931164,12097.21263,8352.856451,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
Arpp21,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [17]:
yCoord[0].head()

,Astro.1,Astro.2,Astro.3,Astro.4,Astro.5,Cacna2d1.Lhx6.Reln,Cacna2d1.Lhx6.Vwa5a,Cacna2d1.Ndnf.Cxcl14,Cacna2d1.Ndnf.Npy,Cacna2d1.Ndnf.Rgs10,...,Sst.Erbb4.Crh,Sst.Erbb4.Rgs10,Sst.Npy.Cort,Sst.Npy.Mgat4c,Sst.Npy.Serpine2,Sst.Npy.Zbtb20,Sst.Pnoc.Calb1.Igfbp5,Sst.Pnoc.Calb1.Pvalb,Sst.Pnoc.Pvalb,Vip.Crh.C1ql1
3110035E14Rik,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0
6330403K07Rik,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,15202.884807,0.0,0.0,...,17954.847027,0.0,0.000000,16867.08335,16073.906284,0.0,17582.159289,0.0,0.0,0.0
Adgrl2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,17428.965287,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0
Aldoc,13824.016583,16376.167983,15121.610374,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0
Arpp21,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [18]:
sample['GeneExp'].shape

(92, 59)